# LlamaIndex: QA over Structured Data (Text-to-SQL Guide)

- Question-Answering (RAG)
  - https://docs.llamaindex.ai/en/stable/use_cases/q_and_a/
- QA over Structured Data (Text-to-SQL Guide)<br>
  - **[Text-to-SQL Guide (Query Engine + Retriever)](https://docs.llamaindex.ai/en/stable/examples/index_structs/struct_indices/SQLIndexDemo/)**

## SETUP

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables (for API key)
load_dotenv()

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable or add it to a .env file")

# Define the model to use
MODEL_GPT = "gpt-4o-mini"

## SETUP (LlamaIndex)

In [2]:
# %pip install llama-index-llms-openai

In [3]:
# !pip install llama-index

In [4]:
import os
import openai

In [5]:
# os.environ["OPENAI_API_KEY"] = "sk-.."
openai.api_key = os.environ["OPENAI_API_KEY"]
# print(openai.api_key)

In [6]:
# import logging
# import sys

# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [7]:
from IPython.display import Markdown, display

## Local Python files (world-capitals)

In [8]:
# Create a dictionary to hold the loaded variables
data_dict = {}

# Read and execute the file content from the data folder
with open('./data/world-capitals-claude.py', 'r') as file:
    exec(file.read(), globals(), data_dict)

# Access the data
capitals = data_dict['rows']

# Example: Find the capital with the largest population
largest_capital = max(capitals, key=lambda x: x["population"])
print(f"The largest capital is {largest_capital['city_name']}, {largest_capital['country']} with {largest_capital['population']:,} people")

The largest capital is Tokyo, Japan with 37,393,000 people


In [9]:
print(len(capitals))
# print(capitals)

193


## Create Database Schema
Use **sqlalchemy**, a popular SQL database toolkit, to create an empty **city_stats** Table

In [10]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
)

In [11]:
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

In [12]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)
metadata_obj.create_all(engine)

## Define SQL Database
Define our SQLDatabase abstraction (a light wrapper around SQLAlchemy)<br>
Add some testing data to our SQL database

In [13]:
from llama_index.core import SQLDatabase
from llama_index.llms.openai import OpenAI

In [14]:
# llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")
llm = OpenAI(temperature=0.1, model="gpt-4o-mini")

In [15]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])

In [16]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])
from sqlalchemy import insert

rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {
        "city_name": "Chicago",
        "population": 2679000,
        "country": "United States",
    },
    {"city_name": "Seoul", "population": 9776000, "country": "South Korea"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [17]:
# view current table
stmt = select(
    city_stats_table.c.city_name,
    city_stats_table.c.population,
    city_stats_table.c.country,
).select_from(city_stats_table)

with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()
    print(results)

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Chicago', 2679000, 'United States'), ('Seoul', 9776000, 'South Korea')]


## Query Index

In [18]:
from sqlalchemy import text

with engine.connect() as con:
    rows = con.execute(text("SELECT city_name from city_stats"))
    for row in rows:
        print(row)

('Chicago',)
('Seoul',)
('Tokyo',)
('Toronto',)


## Part 1: Text-to-SQL Query Engine

In [19]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database, tables=["city_stats"], llm=llm
)
query_str = "Which city has the highest population?"
response = query_engine.query(query_str)

In [21]:
display(Markdown(f"<b>{response}</b>"))

<b>The city with the highest population is Tokyo, with a population of 13,960,000.</b>

## Part 2: Query-Time Retrieval of Tables for Text-to-SQL

In [22]:
from llama_index.core.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine,
)
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import VectorStoreIndex

# set Logging to DEBUG for more detailed outputs
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="city_stats"))
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=1)
)

In [23]:
response = query_engine.query("Which city has the highest population?")
display(Markdown(f"<b>{response}</b>"))

<b>Tokyo has the highest population among all cities, with a population of 13,960,000.</b>

In [24]:
# you can also fetch the raw result from SQLAlchemy!
response.metadata["result"]

[('Tokyo', 13960000)]

In [25]:
# manually set context text
city_stats_text = (
    "This table gives information regarding the population and country of a"
    " given city.\nThe user will query with codewords, where 'foo' corresponds"
    " to population and 'bar'corresponds to city."
)

table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="city_stats", context_str=city_stats_text))
]

## Part 3: Text-to-SQL Retriever

In [26]:
from llama_index.core.retrievers import NLSQLRetriever

# default retrieval (return_raw=True)
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["city_stats"], return_raw=True
)

In [27]:
results = nl_sql_retriever.retrieve(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [28]:
from llama_index.core.response.notebook_utils import display_source_node

for n in results:
    display_source_node(n)

**Node ID:** 94cb6e60-cf32-4fc5-baae-9dd7abe2594b<br>**Similarity:** None<br>**Text:** [('Tokyo', 13960000), ('Seoul', 9776000), ('Toronto', 2930000), ('Chicago', 2679000)]<br>

In [29]:
# default retrieval (return_raw=False)
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["city_stats"], return_raw=False
)

In [30]:
results = nl_sql_retriever.retrieve(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [31]:
# NOTE: all the content is in the metadata
for n in results:
    display_source_node(n, show_source_metadata=True)

**Node ID:** 804b393e-60e2-42e7-8f44-140d255c7876<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Tokyo', 'population': 13960000}<br>

**Node ID:** d426b232-ecba-4c8c-a589-8b8c64c329bb<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Seoul', 'population': 9776000}<br>

**Node ID:** 7188d6ba-3a93-49cd-bf67-69caad620889<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Toronto', 'population': 2930000}<br>

**Node ID:** e68c2b85-4f53-492e-9bbc-56271b9329ee<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Chicago', 'population': 2679000}<br>

## Plug into our RetrieverQueryEngine

In [32]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(nl_sql_retriever)

In [33]:
response = query_engine.query(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [34]:
display(Markdown(f"<b>{response}</b>"))

<b>Return the top 5 cities with the highest population:
1. Tokyo - 13,960,000
2. Seoul - 9,776,000
3. Toronto - 2,930,000
4. Chicago - 2,679,000</b>

In [35]:
print(str(response))

Return the top 5 cities with the highest population:
1. Tokyo - 13,960,000
2. Seoul - 9,776,000
3. Toronto - 2,930,000
4. Chicago - 2,679,000
